# Classificação de Imagens de 03 classes de plantas por tipos de doenças, com Transfer Learning

#### Objetivo: Realizar o Teste de Inferência com imagens de testes de classe, para o cenário do Caso de Teste 04.

##### Classes Consideradas:
###### Grape___Black_rot
###### Grape___Esca_(Black_Measles)
###### Grape___healthy


## Consultas:
##### https://keras.io/applications/
##### https://www.tensorflow.org/tutorials/load_data/images
##### https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
##### https://keras.io/api/data_loading/
##### https://datascience.stackexchange.com/questions/81541/how-to-predict-multiple-images-from-folder-in-python

## Leitura do local da base de dados

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
workdir_path = '/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive' #MODIFICAR!!! #Caminho para o seu workspace
os.chdir(workdir_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
os.path.abspath(os.getcwd())

'/content/drive/.shortcut-targets-by-id/1l60AC27n8nkWWiKY_Crqb8SeOohho5uQ/TrabalhoFinal/DadosTCC/archive'

In [3]:
# Base de imagens para treino com total de 4659 imagens (classes de tipos de platas)
trainpath = workdir_path + '/classeuva/Train'

# Base de imagens para validação com total 1169 imagens (classes de tipos de platas)
validpath = workdir_path + '/classeuva/Valid'

# Base de imagens para validação com total de 30 imagens (classes de tipos de platas)
testpath = workdir_path + '/classeuva/Test'


## Leitura de Bibliotecas

In [4]:
from tensorflow.keras.models import load_model
import argparse
import pickle
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import numpy as np
import logging, os
import sys 
from keras.preprocessing import image
import tensorflow as tf
import math
import operator
from pathlib import Path


## Leitura do modelo do respectivo cenário 06 para o Caso de Teste 04

In [5]:
# Ler modelo treinado gravado para submeter aos dados de testes
modelosalvo = tf.keras.models.load_model('/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive/Inferência-TestesModelos/Caso de Teste 04/VGG19-cenário06/Modelo_Salvo_1/VGG19.h5')


## Carrega as Imagens de Testes Selecionadas

In [6]:
# Prepara variáveis para imagens do tipo Black_rot

Black_rot_path = testpath +'/Black_rot/' # Caminho das imagens

imagens_Black_rot = [] # Inicializa uma lista para carregar as imagens


In [7]:
# Carrega as imagens para do tipo Black_rot
for img in os.listdir(Black_rot_path):                               # Cria um loop
        img = os.path.join(Black_rot_path, img)                        
        img = tf.keras.utils.load_img(img, target_size=(224,224))    # coloca a imagem no shape largura
        img = tf.keras.utils.img_to_array(img)                       # transforma em um array
        img = np.expand_dims(img, axis=0)                            
        imagem_normalizada = np.array(img, dtype="float") / 255.0    # normaliza a imagem dividindo por 255
        imagens_Black_rot.append(imagem_normalizada)                 # carrega a imagem para a lista
        
imagens_Black_rot = np.vstack(imagens_Black_rot)                        

## Previsão Dados de Teste (classe Black-Hot)

In [8]:
# faz a previsao das imagens de teste do tipo Black_rot
dados_previsao_imagens_Black_rot_path = []                            # inicializa uma lista para colocar o reultado da previsao das imagens na lista
nome_arquivos = []                                                    # inicializa uma lista para colocar o nome dos arquivos
for arquivo in os.listdir(Black_rot_path):                            # inicializa um loop (para cada arquivo dentro da pasta)
    pred_result = modelosalvo.predict(imagens_Black_rot)              # faz a previsao de cada arquivo
    dados_previsao_imagens_Black_rot_path.append(pred_result)              # coloca os dados da previsão na lsita
    nome_arquivos.append(arquivo)                                     # coloca o nome do arquivo na lista "nome_arquivos"

1/1 [==============================] - 0s 31ms/step


In [9]:
dados_previsao_imagens_Black_rot_path[0]

array([[6.2479441e-05, 9.9993753e-01, 3.4581579e-16],
       [9.9999797e-01, 1.3565526e-06, 7.4091253e-07],
       [1.0000000e+00, 3.2088249e-08, 1.7930116e-15],
       [1.0000000e+00, 1.3855069e-28, 1.4025896e-33],
       [7.5787735e-01, 2.3096126e-01, 1.1161307e-02],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [7.8406465e-01, 1.9935754e-01, 1.6577810e-02],
       [1.0000000e+00, 7.9142433e-16, 1.2132428e-18],
       [1.0000000e+00, 2.6936605e-13, 3.9316039e-18],
       [1.0000000e+00, 1.0207157e-15, 4.5364533e-16]], dtype=float32)

In [13]:
# Define os labels R:Grape___Black_rot, M:Grape___Esca_(Black_Measles) e H:Grape___healthy' na mesma ordem de classes do modelo na etapa de load.
# Previsão dados de testes selecionados

label_file_path = os.path.join("RMH", 'labels') 

#SORT
top_k = []
pred = [] 
for i in range(len(dados_previsao_imagens_Black_rot_path)):
            ordem = dados_previsao_imagens_Black_rot_path[i][0].argsort()[-len(dados_previsao_imagens_Black_rot_path[i][0]):][::-1]
            top_k.append(ordem)
            top = top_k[i][:10]
            print(nome_arquivos[i])
            for node_id in top:
                planta_string = label_file_path[node_id]
                score = dados_previsao_imagens_Black_rot_path[i][0][node_id]
                print('%s (score = %.5f)' % (planta_string, score))

e6444b5c-2944-46c8-a188-b9fbb366a931___FAM_B.Rot 0655_flipLR.JPG
M (score = 0.99994)
R (score = 0.00006)
H (score = 0.00000)
e95869c1-adf0-40ca-aa7c-14f930d63420___FAM_B.Rot 3537.JPG
M (score = 0.99994)
R (score = 0.00006)
H (score = 0.00000)
e20443d6-a0a5-4750-a239-bdd3d2f6e69e___FAM_B.Rot 0441_flipLR.JPG
M (score = 0.99994)
R (score = 0.00006)
H (score = 0.00000)
efe1eba2-52c3-4399-a851-3095e4d1e86a___FAM_B.Rot 0633.JPG
M (score = 0.99994)
R (score = 0.00006)
H (score = 0.00000)
f0cb2eb6-70cf-4270-9091-0d9ad45eee5c___FAM_B.Rot 3223.JPG
M (score = 0.99994)
R (score = 0.00006)
H (score = 0.00000)
ed40b1ed-928e-4435-b192-8c7b68e1349e___FAM_B.Rot 3131.JPG
M (score = 0.99994)
R (score = 0.00006)
H (score = 0.00000)
e441a2b0-1428-43b4-bbfc-a938f50e4e39___FAM_B.Rot 0499.JPG
M (score = 0.99994)
R (score = 0.00006)
H (score = 0.00000)
f98da630-3128-4702-8e92-60cb024305e5___FAM_B.Rot 5082.JPG
M (score = 0.99994)
R (score = 0.00006)
H (score = 0.00000)
f241eaeb-5b01-4357-840e-3b7e4ff1a539___FAM